In [23]:
#Notebooks for data exploration and cleanup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress 
import warnings

warnings.filterwarnings("ignore")

<h1>Data Load and Segregation of Colorado Data - Substance Abuse Treatment</h1>

In [24]:
# All data downloaded from https://www.samhsa.gov/ -- Substance Abuse and Mental Health Services Administration
#Load the 2018 data
substance_abuse_data_2018 = pd.read_csv("../marijuana_data/TEDS-A-2018-DS0001-bndl-data-tsv/tedsa_puf_2018.csv",  
                                    error_bad_lines=False, na_filter=False, )

In [25]:
colorado_data = substance_abuse_data_2018.loc[substance_abuse_data_2018["STFIPS"] == 8]
colorado_data['CASEID'].count()

83367

In [26]:
#Load the 2015-2017 data and append to end of colorado_data
substance_abuse_data_2017 = pd.read_csv("../marijuana_data/TEDS-A-2015-2017-DS0001-bndl-data-tsv/tedsa_puf_2015_2017.csv",  
                                    error_bad_lines=False, na_filter=False)
data2017 = substance_abuse_data_2017.loc[substance_abuse_data_2017["STFIPS"] == 8]
colorado_data1 = colorado_data.append(data2017, ignore_index = True)
colorado_data1['CASEID'].count()

331803

In [27]:
#Load the 2010-2014 data and append to end of colorado_data
substance_abuse_data_2014 = pd.read_csv("../marijuana_data/TEDS-A-2010-2014-DS0001-bndl-data-tsv/tedsa_puf_2010_2014.csv",  
                                    error_bad_lines=False, na_filter=False)
data2014 = substance_abuse_data_2014.loc[substance_abuse_data_2014["STFIPS"] == 8]
colorado_data2 = colorado_data1.append(data2014, ignore_index = True)
colorado_data2['CASEID'].count()

753016

In [28]:
#Load the 2005-2009 data and append to end of colorado_data
substance_abuse_data_2009 = pd.read_csv("../marijuana_data/TEDS-A-2005-2009-DS0001-bndl-data-tsv/tedsa_puf_2005_2009.csv",  
                                    error_bad_lines=False, na_filter=False)
data2009 = substance_abuse_data_2009.loc[substance_abuse_data_2009["STFIPS"] == 8]
colorado_data_all = colorado_data2.append(data2009, ignore_index = True)
colorado_data_all['CASEID'].count()

1161884

<h2>Data Cleanup</h2>

Data cleanup consisted of removing columns not relevant for the current study and rows with incomplete data. 

Since our analysis focused on how legalization of marijuana impacted substance abuse treatment totals, all personal demographic info (age, sex, employment and veteran status) were not relevant.  In addition, a number of columns are related to payment, other psychiatric issues, or non-marijuana usage patterns.  These columns were also removed.  The final data columns maintained are:

    ADMYR: Year of admission 
    CASEID: Unique identifier for each admission
    SERVICES: Service setting at admission
    SUB1: Substance use at admission (primary)
    SUB2: Substance use at admission (secondary)
    SUB3: Substance use at admission (tertiary)
    MARFLG: Marijuana/hashish reported at admission
    PSYPROB: Co-occurring mental and substance use disorders
    


In [29]:
colorado_data_all.head()

,ADMYR,CASEID,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,DETCRIM,NOPRIOR,PSOURCE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2018,20181674745,8,19740,2,2,7,-9,0,-9,...,0,0,0,0,0,0,8,4,1,2
1,2018,20181598312,8,-9,3,2,7,-9,1,-9,...,0,0,0,0,0,0,8,4,1,2
2,2018,20181669968,8,-9,4,2,7,-9,1,1,...,0,0,0,0,0,0,8,4,1,2
3,2018,20181726629,8,19740,3,2,7,-9,0,1,...,0,0,0,0,0,0,8,4,1,2
4,2018,20181897353,8,19740,3,1,7,-9,5,-9,...,0,0,0,0,0,0,8,4,1,2


In [38]:
clean_colorado_data = colorado_data_all[['ADMYR', 'CASEID', 'SERVICES', 'SUB1', 'SUB2', 'SUB3', 'MARFLG', 'PSYPROB']]

In [39]:
# Save this clean data to a .csv which will be easier to manipulate in analysis notebook
clean_colorado_data.to_csv('data/clean_substance_treatment_data.csv')

In [40]:
clean_colorado_data.count()

ADMYR       1161884
CASEID      1161884
SERVICES    1161884
SUB1        1161884
SUB2        1161884
SUB3        1161884
MARFLG      1161884
PSYPROB     1161884
dtype: int64

Previous analysis revealed no explicitly missing values in the data ('NaT' or 'NaN').  However, several of the fields/columns have a "no value" code of -9 to signify missing data.  Since these values mean the data was not collected or recorded correctly, we will drop those rows unless -9 exists in the SUB2 or SUB3 column.  That will be converted to '1', meaning 'None' on the secondary or tertiary drug use.

In [41]:
clean_colorado_data.loc[clean_colorado_data['SUB2'] == -9, ['SUB2']] = 1
clean_colorado_data.loc[clean_colorado_data['SUB3'] == -9, ['SUB3']] = 1
final_colorado_treatment_data = clean_colorado_data.loc[((clean_colorado_data['SERVICES'] > -1) &
                                                        (clean_colorado_data['SUB1'] > -1) &
                                                        (clean_colorado_data['MARFLG'] > -1) &
                                                        (clean_colorado_data['PSYPROB'] > -1))]
final_colorado_treatment_data['CASEID'].count()

961433

In [42]:
# Save this clean data to a .csv which will be easier to manipulate in analysis notebook
final_colorado_treatment_data.to_csv('data/clean_substance_treatment_data.csv')